In [1]:
import os
import io
import sys
cwd = os.getcwd()
sys.path.insert(0, cwd)
from module.importData import importData
from module.toWordList import toWordList
from module.steamingWiki import steamingWiki
from module.makeModelGensim import makeModelGensim
from module.toVectore import toVectore
#from module.modelLSTM import modelLSTM
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, cohen_kappa_score
import pandas as pd
import numpy as np
import gensim
import keras
from keras.models import load_model

C:\Users\s2\Anaconda3\envs\tgpu\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
C:\Users\s2\Anaconda3\envs\tgpu\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
np.set_printoptions(threshold=sys.maxsize)
pd.get_option("display.max_rows", 1000)
pd.set_option('display.max_columns', 1000)

In [3]:
wikiSource			= 'idwiki'
answerData			= 'DataAnswerExam_SMP.csv'
questionData		= 'qes.csv'
dirData				= cwd+'/data/'
corpusInput			= wikiSource+'.bz2'
wikiOutput			= wikiSource+'.txt'
fileExtension		= 'bin'
trainingAlgoritm	= 0
numDimension		= 200
modelOutput			= wikiSource+'_word2vec_'+str(numDimension)+'_'+str(trainingAlgoritm)+'.'+fileExtension

In [4]:
dAnswer, dQuestion = importData(answer= dirData+answerData, question= dirData+questionData).openData()

if not(os.path.exists(dirData+modelOutput)):
	if not(os.path.exists(dirData+wikiOutput)):
		steamingWiki(corpusInput=corpusInput, wikiOutput=wikiOutput).execute()
	makeModelGensim(wikiOutput=wikiOutput, modelOutput=modelOutput, numDimension=numDimension, trainingAlgoritm=trainingAlgoritm).execute()

if fileExtension != 'bin':
	model = gensim.models.word2vec.Word2Vec.load(dirData+modelOutput)
else:
	model = gensim.models.KeyedVectors.load_word2vec_format(dirData+modelOutput, unicode_errors='ignore')


In [5]:
crossVal = KFold(n_splits=2, random_state=True, shuffle=True)

count = 1

In [6]:
dAnswer.head()

,Essay_id,Answer,Score,TrueAnswer
0,1,Alumni; ; Kelebihan; Permisalan; Tutorial; Im...,3,lulusan : alumnus; rencana : agenda; keuntunga...
120,1,lulusan : alumnus; rencana : agenda; keuntunga...,5,lulusan : alumnus; rencana : agenda; keuntunga...
365,1,alumni; argument; hasil; example; teknik; imaj...,4,lulusan : alumnus; rencana : agenda; keuntunga...
650,1,alumnus; agenda; laba; sampel; teknik; imajina...,5,lulusan : alumnus; rencana : agenda; keuntunga...
125,1,lulusan : alumnus; rencana : agenda; keuntunga...,5,lulusan : alumnus; rencana : agenda; keuntunga...


In [7]:
xtrain = dAnswer.loc[:,['Answer','TrueAnswer']]

In [8]:
xtrain.head()

,Answer,TrueAnswer
0,Alumni; ; Kelebihan; Permisalan; Tutorial; Im...,lulusan : alumnus; rencana : agenda; keuntunga...
120,lulusan : alumnus; rencana : agenda; keuntunga...,lulusan : alumnus; rencana : agenda; keuntunga...
365,alumni; argument; hasil; example; teknik; imaj...,lulusan : alumnus; rencana : agenda; keuntunga...
650,alumnus; agenda; laba; sampel; teknik; imajina...,lulusan : alumnus; rencana : agenda; keuntunga...
125,lulusan : alumnus; rencana : agenda; keuntunga...,lulusan : alumnus; rencana : agenda; keuntunga...


In [9]:
#keras.utils.to_categorical(dAnswer.loc[:,['Score']], num_classes=6)

In [10]:
class modelLSTM(object):
	from keras.models import Sequential, Model
	from keras.layers import LSTM, Dense, Dropout, Bidirectional, Lambda, Dot, Subtract
	import keras.backend as K
	from keras import optimizers

	def biSiamenseModel(self, inputD, euclidean=False):

		from keras.layers import Input, concatenate, subtract, dot
		from keras import backend as K

		def euclidean_distance(vects):
			x, y = vects
			sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
			return K.sqrt(K.maximum(sum_square, K.epsilon()))

		def manhattan_distance(vects):
			x, y = vects
			return K.sum((K.abs(x - y)), axis=1, keepdims=True)

		def exponent_neg_manhattan_distance(vects):
			x, y = vects
			return K.exp(-K.sum(K.abs(x-y), axis=1, keepdims=True))


		MA = Input(shape= inputD, dtype="float32")
		MB = Input(shape= inputD, dtype="float32")

		x = self.Bidirectional(self.LSTM(200, dropout=0.2, recurrent_dropout=0.2, return_sequences=True), input_shape=inputD)(MA)
		x = self.Bidirectional(self.LSTM(150, dropout=0.2, recurrent_dropout=0.2))(x)

		y = self.Bidirectional(self.LSTM(200, dropout=0.2, recurrent_dropout=0.2, return_sequences=True), input_shape=inputD)(MB)
		y = self.Bidirectional(self.LSTM(150, dropout=0.2, recurrent_dropout=0.2))(y)
		# a2 = (self.Lambda(function=lambda a: euclidean_distance(a),output_shape=lambda a: (a,1)))([x,y])

		if euclidean:
			a2 = (self.Lambda(function=lambda a: euclidean_distance(a),output_shape=lambda a: (a,1)))([x,y])
		else:
			#a2 = concatenate([x,y])
			a2 = self.Subtract()([x,y])
		
		out = self.Dense(1, activation='relu')(a2)

		model = self.Model(inputs=[MA, MB], outputs=out)
		sgd = self.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
		model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['mae'])

		model.summary()

		return model	

In [11]:
for dx, dy in crossVal.split(dAnswer):

	trainSAnswer = []
	trainTAnswer = []
	testSAnswer = []
	testTAnswer = []
	
	print("\n--------Fold {}--------\n".format(count))
	train, test= dAnswer.iloc[dx], dAnswer.iloc[dy]
	
	
	xtrain = train.loc[:,['Answer', 'TrueAnswer']]
	xtest = test.loc[:,['Answer', 'TrueAnswer']]
	ytrain = train.loc[:,['Score']].values
	ytest = test.loc[:,['Score']].values
	#ytraincat, idtrain = np.unique(train.loc[:,['Score']].values, return_inverse=True)
	#ytrain = keras.utils.to_categorical(idtrain, len(ytraincat))
	#ytestcat, idtest = np.unique(test.loc[:,['Score']].values, return_inverse=True)
	#ytest = keras.utils.to_categorical(idtest, len(ytestcat))
	#ytrain = keras.utils.to_categorical(train.loc[:,['Score']].values, num_classes=6)
	#ytest = keras.utils.to_categorical(test.loc[:,['Score']].values, num_classes=6)
	
	# bagian ini biar universal aja.
	
	[trainSAnswer.append(toWordList().sentenceToWordList(a1[0], changeNumber2Word= True)) for a1 in xtrain.loc[:,['Answer']].values]
	[trainTAnswer.append(toWordList().sentenceToWordList(a1[0], changeNumber2Word= True)) for a1 in xtrain.loc[:,['TrueAnswer']].values]
	[testSAnswer.append(toWordList().sentenceToWordList(a1[0], changeNumber2Word= True)) for a1 in xtest.loc[:,['Answer']].values]
	[testTAnswer.append(toWordList().sentenceToWordList(a1[0], changeNumber2Word= True)) for a1 in xtest.loc[:,['TrueAnswer']].values]
	
	vtrainSAnswer, vtrainTAnswer = toVectore(essays = trainSAnswer, trueAnswer=trainTAnswer, model = model, numFeature= numDimension, average=False, distance=True).changeToVector()
	vtestSAnswer, vtestTAnswer = toVectore(essays = testSAnswer, trueAnswer=testTAnswer, model = model, numFeature= numDimension, average=False, distance=True).changeToVector()
	modelNetwork = modelLSTM().biSiamenseModel(inputD=(vtrainSAnswer.shape[1], vtrainSAnswer.shape[2]))
	modelNetwork.fit([vtrainSAnswer, vtrainTAnswer], ytrain, batch_size=100, epochs=10)
	pred = modelNetwork.predict([vtestSAnswer, vtestTAnswer])


--------Fold 1--------

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 180, 200)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 180, 200)     0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 180, 400)     641600      input_1[0][0]                    
_______________________________________________________________________________________________

In [12]:
dfa = pd.DataFrame()

In [13]:
cohen_kappa_score(ytest, np.around(pred), weights='quadratic')

0.6830400345757426

In [14]:
mean_squared_error(ytest, np.around(pred))

0.8871794871794871

In [15]:
mean_absolute_error(ytest, np.around(pred))

0.5743589743589743

In [16]:
ytest

array([[5],
       [4],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [3],
       [5],
       [5],
       [3],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [5],
       [4],
       [5],
       [5],
       [5],
       [5],
       [3],
       [5],
       [5],
       [5],
       [5],
       [5],
       [2],
       [2],
       [5],
       [2],
       [5],
       [2],
       [5],
       [5],
       [5],
       [2],
       [2],
       [5],
       [5],
       [5],
       [5],
       [2],
       [2],
       [5],
       [5],
       [5],
       [2],
       [5],
    

In [17]:
pred

array([[5.375099 ],
       [4.046815 ],
       [5.5191174],
       [5.5527253],
       [5.292536 ],
       [5.224403 ],
       [5.5021586],
       [5.6024194],
       [5.433441 ],
       [5.457258 ],
       [5.494593 ],
       [4.6552052],
       [5.153135 ],
       [5.4933715],
       [4.7852006],
       [5.2595716],
       [5.6024194],
       [5.420472 ],
       [5.166968 ],
       [5.5693226],
       [5.6024194],
       [5.5592256],
       [5.6340246],
       [4.8029966],
       [5.448654 ],
       [5.6024194],
       [5.6024194],
       [5.4537477],
       [5.479692 ],
       [5.6329923],
       [5.6024194],
       [5.6024194],
       [5.6024194],
       [5.514079 ],
       [5.406247 ],
       [5.4707274],
       [5.3940363],
       [5.4619923],
       [5.2864294],
       [5.1142993],
       [5.6024194],
       [5.4943447],
       [5.6024194],
       [5.2521195],
       [5.2737117],
       [5.2567887],
       [5.4697413],
       [5.6024194],
       [5.125488 ],
       [5.6024194],
